# <center> IBM Data Science Professional Certificate Capstone Project </center>

This notebook will be used for my Capstone Project of IBM Data Science Professional Certificate course powered by Coursera

## <center>The Battle of Neighborhoods</center>

In [1]:
#importing needed libraries
import pandas as pd
import numpy as np

import types

from botocore.client import Config
import ibm_boto3

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### 4th week Final Assignment

# <center> Toward a better city </center>

## 1. Introduction - Business problem

### Question  
  
  #### Which can be considered good investments  in order to improve the quality level of the tough Neighborhoods of the city?

### Context 
  
  **The public administration of the City of Chicago** (**the customer/stakeholder**) is going to build its plan for future development of the city.  
  The view of administrators is the rebalancing of different areas of the city in terms of life quality and perception.  
  A better Neighborhood doesn't mean a positive improve only of their habitants, but for all the city: young people, growing in a favorable environment, could be stimuled to gain a higher education and have a perspective for higher salaries and life quality/style, a redeveloped Neighborhood can attract more people and businesses, properties can rise their values, etc...and all these improvements can bring to higher taxes paid in return to the city, safer Neighborhood where the need of public intervent is reduced, etc.. (returns on investment).  
    
For this reason the Pubblic Administration is willing to better understand how to use their budget and a Data Scientists team has been asked to analyze and suggest potential investements.
  

### Identified business requirements
  
The Data Scientists team has identified the following requirements:  
  
* Evaluating if venues or activities help in the redevelopment process of a tough Neighborhood or not
* Evaluating which venue or activity categories should be stimulated by the mean of public investments
* Deploying a priority chart for venue categories that improve a Neighborhood
* Evaluating if maybe some venue category can have the opposite effect, i.e. they promote degradation of the Neighborhood 

## 2. Data

In order to perform the analysis, Data Scientists team will use:  
1. **Chicago socioeconomic dataset**  

   This dataset is available from the website of the City of Chicago (https://data.cityofchicago.org/Health-Human-Services/Selected-socioeconomic-indicators-by-neighborhood/i9hv-en6g) and contains a selection of six socioeconomic indicators of public health significance and a **“hardship index”** by Chicago community area. The indicators are:  
    - the percent of occupied housing units with more than one person per room (i.e., crowded housing);  
    - the percent of households living below the federal poverty level;  
    - the percent of persons in the labor force over the age of 16 years that are unemployed;  
    - the percent of persons over the age of 25 years without a high school diploma; the percent of the population under 18 or over 64 years of age (i.e., dependency);  
    - and per capita income;  
    - hardship index.  

   Indicators for Chicago as a whole are provided in the final row of the table.  
   

2. **Chicago reported crimes dataset**  

    This dataset (available at https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2) reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present and contains the following fields:
    - ID: unique identifier for the record;
    - Case Number: the Chicago Police Department RD Number (Records Division Number), which is unique to the incident;
    - Date: date when the incident occurred. this is sometimes a best estimate;
    - Block: the partially redacted address where the incident occurred, placing it on the same block as the actual address;
    - IUCR: the Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description;
    - Description: the secondary description of the IUCR code, a subcategory of the primary description;
    - Location Description: description of the location where the incident occurred;
    - Arrest: indicates whether an arrest was made;
    - Domestic: indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act;
    - Beat: indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74;
    - District: indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r;
    - Ward: the ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76;
    - Community Area: indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6;
    - FBI Code: indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html;
    - X Coordinate: the x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block;
    - Y Coordinate: the y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block;
    - Year: year the incident occurred;
    - Updated On: date and time the record was last updated;
    - Latitude: the latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block;
    - Longitude: the longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block;
    - Location: the location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.  
      
    These two databases will be mainly used to cluster and label Chicago Neighborhoods considering their:  
      - hardship index;
      - crime frequency;
      - degree of crime
    
    Furthermore, as Crime dataset includes reported crime since 2001 and is very big, it will be sliced in order to use only data referred to reported crimes since 2015.
   

3. **Foursquare venues and checkins API**  

    The Team will use Foursquare API in order to discover venues in a specific Neighborhood or at a specific location. In addition checkins will be eventually used to analyze the popularity of specific venues.

Let's give an overview of dataset structures

In [4]:
def __iter__(self): return 0

body = client_de7790dfb63141b0ae0ffdfc73f2f446.get_object(Bucket='courseracapstone-donotdelete-pr-cvzsktymx4cuiu',Key='Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
dfSE = pd.read_csv(body)

print(dfSE.shape)
dfSE.head()


(78, 9)


,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [5]:

body = client_de7790dfb63141b0ae0ffdfc73f2f446.get_object(Bucket='courseracapstone-donotdelete-pr-cvzsktymx4cuiu',Key='Crimes_-_2001_to_present.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
dfCrimes = pd.read_csv(body)

print(dfCrimes.shape)
dfCrimes.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(7130308, 22)


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11034701,JA366925,01/01/2001 11:00:00 AM,016XX E 86TH PL,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,8.0,45.0,11,NaN,NaN,2001,08/05/2017 03:50:08 PM,NaN,NaN,NaN
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN


### end of 4th week Final Assignment